In [1]:
from itertools import chain
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from moviepy import VideoFileClip, concatenate_videoclips
from scipy.optimize import curve_fit

In [2]:
def frap_plot(filepath: Path):
    try:
        df = pd.read_csv(filepath, encoding='utf-16', index_col=0)
    except pd.errors.ParserError:
        df = pd.read_csv(filepath, encoding='utf-16', index_col=0, skiprows=1)

    # print(f'row count: {df.shape[0]}')

    # Clean the df
    df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
    df.columns = df.columns.str.replace(r'\s*\[\]\s*', '', regex=True)  # Remove empty brackets
    df = df.dropna(axis=1, how='all')  # Drop columns that are completely empty

    # df = df.dropna(axis=0, how='all')  # Drop rows that are completely empty
    df = df.loc[:, (df != 0).mean(axis=0) > 0.1]  # Drop columns with more than 90% zeros
    d_rows = df.shape[0] - df.shape[0]
    # print(f'row count after cleaning: {df.shape[0]}, dropped {d_rows} rows')

    # Scatter plot the cleaned data using Plotly
    fig = go.Figure()

    for column in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df[column],
                mode='lines',
                name=column,
            ),
        )

    fig.update_layout(
        title=filepath.stem,
        xaxis_title='Time (s)',
        yaxis_title='Intensity (a.u.)',
        legend_title='Legend',
        template='plotly_white',
    )

    fig.show()

In [3]:
# get all the names of the fraps

base_path = Path().absolute().parent.parent.parent / 'data/viral_condensates'
all_csvs = list(base_path.glob('**/*.csv'))
all_fraps = [
    filepath.parent
    for filepath in all_csvs
]

In [4]:
print(f'Found {len(all_fraps)} fraps')
[i.as_posix() for i in all_fraps]

Found 83 fraps


['c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP01',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP02',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP03',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP04',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP05',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP06',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP01',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP02',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP03',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP04',
 'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP05',
 'c:/Code/data/viral_condensates/hsv1/ul21/UL21 GFP FRAP01',
 'c:/Code/data/viral_condensates/hsv1/ul21/UL21 GFP FRAP02',
 'c:/Code/data/viral_condensates/hsv1/ul21/UL21 GFP FRAP03',
 'c:/Code/data/viral_condensates/hsv1/ul21/UL21 GFP FRAP04',
 'c:/Code/data/viral_condensates/hsv1/ul21/UL21 GFP FRAP05',

In [5]:
# concatenate all movies in the same folder

for frap in all_fraps:
    output_path = frap / f'{frap.name}.avi'
    if output_path.exists():
        print(f"'{output_path.as_posix()}' already exists, skipping...")
        continue

    clips = [
        VideoFileClip(filepath.as_posix())
        for filepath in chain(frap.glob('*.avi'),
                              frap.glob('*.mp4'))
    ]
    if not clips:
        print(f"No video files found in '{frap.as_posix()}'")
        continue

    merged_clip = concatenate_videoclips(clips)
    merged_clip.write_videofile(output_path.as_posix(), codec='libx264', audio_codec='aac')
    print(f"Merged {len(clips)} clips into '{output_path.as_posix()}'")

'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP01/UL14 GFP FRAP01.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP02/UL14 GFP FRAP02.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP03/UL14 GFP FRAP03.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP04/UL14 GFP FRAP04.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP05/UL14 GFP FRAP05.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP FRAP06/UL14 GFP FRAP06.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP01/UL14 GFP HSV1 FRAP01.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP02/UL14 GFP HSV1 FRAP02.avi' already exists, skipping...
'c:/Code/data/viral_condensates/hsv1/ul14/UL14 GFP HSV1 FRAP03/UL14 GFP HSV1 FRAP03.avi' already exists, skipping...
'c:/Cod

In [ ]:
# plot all csvs in the same folder

for file in all_csvs:
    frap_plot(file)